In [5]:
import pandas as pd
import numpy as np

def merge_datasets_for_ml():
    # Read the weather data
    print("Reading weather data...")
    weather_df = pd.read_csv('weather_processing_w_countries.csv')
    
    # Read the leptospirosis data
    print("Reading leptospirosis data...")
    lepto_df = pd.read_csv('health_data_cleaned.csv')
    
    # Clean leptospirosis data
    print("Preparing leptospirosis data...")
    lepto_df['Values_Updated'] = lepto_df['Values_Updated'].replace('-', np.nan)
    lepto_df['Values_Updated'] = pd.to_numeric(lepto_df['Values_Updated'], errors='coerce')
    lepto_df = lepto_df.rename(columns={
        'Time': 'Year',
        'RegionCode': 'Country Code',
        'Values_Updated': 'Leptospirosis_Rate'
    })
    
    # Keep only distinct values per (Country Code, Year)
    print("Removing duplicates from leptospirosis data...")
    lepto_df = lepto_df.groupby(['Year', 'Country Code'], as_index=False).agg({
        'Leptospirosis_Rate': 'mean',  # If multiple values exist, take the average
        'RegionName': 'first'  # Keep the first occurrence of the region name
    }).rename(columns={'RegionName': 'Country Name'})

    # Merge with weather data
    print("Merging datasets...")
    weather_df['Year'] = weather_df['Year'].astype(int)

    # Remove latitude and longitude columns
    weather_df = weather_df.drop(columns=['Latitude', 'Longitude'], errors='ignore')

    # Merge datasets on Year & Country Code
    merged_df = pd.merge(weather_df, lepto_df, on=['Year', 'Country Code'], how='left')

    # Fix duplicate country name columns
    merged_df = merged_df.rename(columns={'Country Name_x': 'Country Name'}).drop(columns=['Country Name_y'], errors='ignore')

    # Drop rows where Leptospirosis Rate is missing
    merged_df = merged_df.dropna(subset=['Leptospirosis_Rate'])

    # Ensure only one Leptospirosis Rate per (Country Code, Year)
    print("Ensuring unique Leptospirosis Rate per Country-Year...")
    merged_df = merged_df.groupby(['Year', 'Country Code'], as_index=False).agg({
        'T2M': 'mean',  # Average temperature
        'D2M': 'mean',  # Average dew point
        'TP': 'sum',  # Total precipitation
        'Leptospirosis_Rate': 'mean',  # Average if multiple exist
        'Country Name': 'first'  # Keep first occurrence
    })

    # Convert temperature and humidity features
    merged_df['Temperature_Celsius'] = merged_df['T2M'] - 273.15  # Convert Kelvin to Celsius
    merged_df['Dew_Point_Celsius'] = merged_df['D2M'] - 273.15
    merged_df['Relative_Humidity'] = 100 * (np.exp((17.625 * merged_df['Dew_Point_Celsius']) / 
                                                  (243.04 + merged_df['Dew_Point_Celsius'])) / 
                                          np.exp((17.625 * merged_df['Temperature_Celsius']) / 
                                                (243.04 + merged_df['Temperature_Celsius'])))
    
    # Save to CSV
    print("Saving merged dataset...")
    merged_df.to_csv('ml_data.csv', index=False)
    
    # Print summary
    print("\nMerged Dataset Summary:")
    print(f"Total rows: {len(merged_df)}")
    print("\nFeatures available for ML:")
    for col in merged_df.columns:
        print(f"- {col}")
    
    # Print some statistics
    print("\nData points per country:")
    print(merged_df['Country Name'].value_counts())

    print("\nData points per year:")
    print(merged_df['Year'].value_counts().sort_index())
    
    return merged_df

if __name__ == "__main__":
    try:
        merged_data = merge_datasets_for_ml()
        print("\nData merge completed successfully!")
    except Exception as e:
        print(f"Error occurred: {e}")


Reading weather data...
Reading leptospirosis data...
Preparing leptospirosis data...
Removing duplicates from leptospirosis data...
Merging datasets...
Ensuring unique Leptospirosis Rate per Country-Year...
Saving merged dataset...

Merged Dataset Summary:
Total rows: 429

Features available for ML:
- Year
- Country Code
- T2M
- D2M
- TP
- Leptospirosis_Rate
- Country Name
- Temperature_Celsius
- Dew_Point_Celsius
- Relative_Humidity

Data points per country:
Country Name
Austria           17
Italy             17
Slovakia          17
Slovenia          17
Sweden            17
Romania           17
Poland            17
Netherlands       17
Malta             17
Latvia            17
Luxembourg        17
Belgium           17
Lithuania         17
Hungary           17
Finland           17
Spain             17
Greece            17
Estonia           17
Denmark           17
Germany           17
Czechia           17
Cyprus            17
Bulgaria          17
France            14
United Kingdom    